In [8]:
"""Integration test: generate a DecisionDAG via an LLM for a Massachusetts building stock.

Run with:
    uv run --env-file .env python tests/test_decision_dag/test_llm_integration.py
"""

from __future__ import annotations

from epinterface.sbem.decision_dag.agent import generate_dag
from epinterface.sbem.decision_dag.executor import DAGExecutor
from epinterface.sbem.decision_dag.fields import (
    UserFieldSet,
)
from epinterface.sbem.decision_dag.validation import validate_dag_structure

SAMPLE_ROWS = [
    {
        "income": 45000,
        "year_built": 1925,
        "building_typology": "single_family_detached",
        "weatherization_status": "not_weatherized",
        "last_renovation": "never",
    },
    {
        "income": 120000,
        "year_built": 2018,
        "building_typology": "single_family_detached",
        "weatherization_status": "unknown",
        "last_renovation": "2010_present",
    },
    {
        "year_built": 1955,
        "building_typology": "small_multifamily_2_4",
        "weatherization_status": "weatherized",
    },
    {
        "building_typology": "large_multifamily_20_plus",
    },
    {},
]

DIRECT_VALUES = {
    "WWR": 0.15,
    "F2FHeight": 3.0,
    "NFloors": 2,
    "Width": 10.0,
    "Depth": 12.0,
    "Rotation": 0.0,
    "EPWURI": "https://energyplus-weather.s3.amazonaws.com/north_and_central_america_wmo_region_4/USA/MA/USA_MA_Boston-Logan.Intl.AP.725090_TMYx.2007-2021.epw",
}


async def main(FIELD_SET: UserFieldSet, dag_output_path: str):
    """Run the full integration test."""
    dag = await generate_dag(FIELD_SET, model="openai:gpt-5.2-chat-latest")

    errors = validate_dag_structure(
        dag, field_set=FIELD_SET, require_full_coverage=True
    )
    if errors:
        print(f"  Validation found {len(errors)} issue(s):")
        for e in errors:
            print(f"    - {e}")
    else:
        print("  DAG passed all structural validation checks.")

    executor = DAGExecutor(dag)
    for i, row in enumerate(SAMPLE_ROWS):
        print(f"\n  Row {i + 1}: {row or '(empty)'}")
        result = executor.execute(row)
        with open(f"result_{i}.json", "w") as f:
            f.write(result.model_dump_json(indent=2))

        highlights = {
            k: result.assignments[k]
            for k in [
                "HeatingFuel",
                "HeatingSystemCOP",
                "InfiltrationACH",
                "FacadeStructuralSystem",
                "FacadeCavityInsulationRValue",
                "WindowUValue",
            ]
            if k in result.assignments
        }
        print(f"    Key values: {highlights}")

    # print("\n--- Attempting FlatModel Construction (Row 1) ---")
    # try:
    #     flat_model = executor.execute_to_flat_model(
    #         SAMPLE_ROWS[0], direct_values=DIRECT_VALUES
    #     )
    #     print("  FlatModel created successfully!")
    #     print(f"  HeatingFuel={flat_model.HeatingFuel}")
    #     print(f"  InfiltrationACH={flat_model.InfiltrationACH}")
    #     print(f"  FacadeStructuralSystem={flat_model.FacadeStructuralSystem}")
    #     print(
    #         f"  FacadeCavityInsulationRValue={flat_model.FacadeCavityInsulationRValue}"
    #     )
    #     print(f"  WindowUValue={flat_model.WindowUValue}")
    # except Exception as exc:
    #     print(f"  FlatModel construction failed: {exc}")

    # print("\n--- Serialized DAG (JSON) ---")
    dag_json = dag.model_dump_json(indent=2)
    from pathlib import Path

    Path(dag_output_path).write_text(dag_json)


In [ ]:
from epinterface.sbem.decision_dag.schema_utils import get_flat_model_schema_description

print(get_flat_model_schema_description())

# FlatModel Parameters

## Equipment Schedule
  - EquipmentBase (float) [ge=0, le=1] default=PydanticUndefined -- Overnight baseload fraction for equipment schedule [0-1].
  - EquipmentAMInterp (float) [ge=0, le=1] default=PydanticUndefined -- Morning peak interpolation fraction for equipment schedule (6-9am) [0-1].
  - EquipmentLunchInterp (float) [ge=0, le=1] default=PydanticUndefined -- Lunch period interpolation fraction for equipment schedule (12-1pm) [0-1].
  - EquipmentPMInterp (float) [ge=0, le=1] default=PydanticUndefined -- Evening peak interpolation fraction for equipment schedule (6-8pm) [0-1].
  - EquipmentWeekendPeakInterp (float) [ge=0, le=1] default=PydanticUndefined -- Weekend peak interpolation fraction for equipment schedule [0-1].
  - EquipmentSummerPeakInterp (float) [ge=0, le=1] default=PydanticUndefined -- Summer peak interpolation fraction for equipment schedule [0-1].

## Lighting Schedule
  - LightingBase (float) [ge=0, le=1] default=PydanticUndefined -- Overn

In [9]:
import yaml

from epinterface.sbem.decision_dag.visualize import visualize_dag_from_files

fieldset_path = "working_user_field_set.yaml"
outpath = "working_dag.json"
md_viz_path = "working_dag.md"
with open(fieldset_path) as f:
    FIELD_SET = UserFieldSet.model_validate(yaml.safe_load(f))

await main(FIELD_SET, outpath)

visualize_dag_from_files(
    dag_path=outpath,
    field_set_path=fieldset_path,
    output_path=md_viz_path,
)


  DAG passed all structural validation checks.

  Row 1: {'income': 45000, 'year_built': 1925, 'building_typology': 'single_family_detached', 'weatherization_status': 'not_weatherized', 'last_renovation': 'never'}
    Key values: {'HeatingFuel': 'FuelOil', 'HeatingSystemCOP': 0.82, 'InfiltrationACH': 1.0, 'FacadeStructuralSystem': 'woodframe', 'FacadeCavityInsulationRValue': 0.5, 'WindowUValue': 2.8}

  Row 2: {'income': 120000, 'year_built': 2018, 'building_typology': 'single_family_detached', 'weatherization_status': 'unknown', 'last_renovation': '2010_present'}
    Key values: {'HeatingFuel': 'Electricity', 'HeatingSystemCOP': 3.0, 'InfiltrationACH': 0.5, 'FacadeStructuralSystem': 'woodframe', 'FacadeCavityInsulationRValue': 4.0, 'WindowUValue': 1.9}

  Row 3: {'year_built': 1955, 'building_typology': 'small_multifamily_2_4', 'weatherization_status': 'weatherized'}
    Key values: {'HeatingFuel': 'FuelOil', 'HeatingSystemCOP': 0.82, 'InfiltrationACH': 0.5, 'FacadeStructuralSystem': 

'# Decision DAG Visualization\n\n> Decision DAG for Massachusetts (IECC 5A) residential buildings. Organized into independent subgraphs for schedules, thermostats, envelope (era-based with weatherization refinement), windows, HVAC & DHW, and space use/ventilation. Missing data is handled with sensible defaults (e.g., unknown weatherization treated as not weatherized unless recent renovation 2010_present). All required FlatModel parameters are assigned on every execution path via base components and refinements.\n\n## Summary\n\n- **Components**: 17\n- **Nodes**: 25\n  - Condition: 7, Assignment: 1, ComponentRef: 17\n- **Entry points**: 5 (`schedule_by_typology`, `thermostat_node`, `envelope_by_era`, `hvac_by_era`, `space_use_defaults`)\n\n## Graph\n\n```mermaid\nflowchart TD\n    schedule_by_typology{{ENTRY: Schedules by typology.}}\n    schedule_by_typology -->|"building_typology in [\'single_family_detached\', ..."| apply_schedule_sf\n    schedule_by_typology -->|"building_typology i

In [ ]:
executor